In [6]:
# Setup environment
%run 0-Environment_Setup.ipynb

Stored 's3_datalake_path_csv' (str)
Stored 'local_data_path_csv' (str)
Stored 's3_datalake_path_parquet' (str)


In [23]:
from sklearn import preprocessing

# Set session variables
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_session.region_name
bucket = sess.default_bucket()

train_prefix = "store-sales-forecasting/train"
test_prefix = "store-sales-forecasting/test"
val_prefix = "store-sales-forecasting/val"

In [24]:
# Create a temporary directory for saving data
local_path = f"{os.getcwd()}/tmp"
if not os.path.exists(local_path):
    os.makedirs(local_path)

In [25]:
# Pull the data from the feature store sorted by date and then store number
sales_features_store_df = get_store_dataset_from_offline_feature_group_date_sort(store_sales_feature_group)
sales_features_store_df.head()

Running 
    SELECT *
    FROM
        "store_sales_feature_group_offline_1728878780"
    ORDER BY
        date ASC, store_nbr ASC
    


,date,store_nbr,sales,oil,onpromotion,is_holiday,city,state,cluster,year,...,month_sin,day_cos,day_sin,dow_cos,dow_sin,sales_record_id,event_time,write_time,api_invocation_time,is_deleted
0,2013-01-01,1,0.0,93.14,0,1,18,12,13,2013,...,0.5,0.97953,0.201299,0.62349,0.781831,2013-01-01:1,1.728879e+09,2024-10-14 04:11:58.235,2024-10-14 04:06:44.000,False
1,2013-01-01,2,0.0,93.14,0,1,18,12,13,2013,...,0.5,0.97953,0.201299,0.62349,0.781831,2013-01-01:2,1.728879e+09,2024-10-14 04:11:58.188,2024-10-14 04:06:45.000,False
2,2013-01-01,3,0.0,93.14,0,1,18,12,8,2013,...,0.5,0.97953,0.201299,0.62349,0.781831,2013-01-01:3,1.728879e+09,2024-10-14 04:11:58.119,2024-10-14 04:06:45.000,False
3,2013-01-01,4,0.0,93.14,0,1,18,12,9,2013,...,0.5,0.97953,0.201299,0.62349,0.781831,2013-01-01:4,1.728879e+09,2024-10-14 04:11:57.992,2024-10-14 04:06:45.000,False
4,2013-01-01,5,0.0,93.14,0,1,21,14,4,2013,...,0.5,0.97953,0.201299,0.62349,0.781831,2013-01-01:5,1.728879e+09,2024-10-14 04:11:58.307,2024-10-14 04:06:45.000,False


In [26]:
# Drop miscellaneous info
sales_features_store_df.drop(columns=["sales_record_id", "event_time", "write_time", "api_invocation_time", "is_deleted"], inplace=True)

In [27]:
sales_features_store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90936 entries, 0 to 90935
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                90936 non-null  object 
 1   store_nbr           90936 non-null  int64  
 2   sales               90936 non-null  float64
 3   oil                 90936 non-null  float64
 4   onpromotion         90936 non-null  int64  
 5   is_holiday          90936 non-null  int64  
 6   city                90936 non-null  int64  
 7   state               90936 non-null  int64  
 8   cluster             90936 non-null  int64  
 9   year                90936 non-null  int64  
 10  month               90936 non-null  int64  
 11  day                 90936 non-null  int64  
 12  dow                 90936 non-null  int64  
 13  city-state          90936 non-null  object 
 14  city-state-cluster  90936 non-null  object 
 15  hash_0              90936 non-null  float64
 16  hash

In [28]:
# Perform feature selection
def get_store_features(row):
    return [
      row["sales"], 
      row["oil"], 
      row["onpromotion"],
      row["is_holiday"], 
      row["hash_0"], 
      row["hash_1"], 
      row["hash_2"], 
      row["hash_3"], 
      row["hash_4"], 
      row["hash_5"], 
      row["hash_6"], 
      row["hash_7"], 
      row["hash_8"], 
      row["hash_9"], 
      row["month_cos"],
      row["month_sin"],
      # row["day_cos"],
      # row["day_sin"],
      row["dow_cos"],
      row["dow_sin"]
]

# Set the number of continuous features being used
num_continuous_features = 3

sales_features_store_df.head().apply(get_store_features, axis=1)

0    [0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 1.0, 0.0, 0....
1    [0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 1.0, 0.0, 0....
2    [0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, -1...
3    [0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0....
4    [0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0....
dtype: object

In [29]:
# Apply to the whole dataframe
sales_features_store_df["features"] = sales_features_store_df.apply(get_store_features, axis=1)

In [30]:
# Drop everything except the store number, date, and selected features
drop_columns = [col for col in sales_features_store_df.columns if col not in ["date", "store_nbr", "features"]]
sales_features_store_df.drop(columns=drop_columns, inplace=True)
sales_features_store_df.head()

,date,store_nbr,features
0,2013-01-01,1,"[0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 1.0, 0.0, 0...."
1,2013-01-01,2,"[0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 1.0, 0.0, 0...."
2,2013-01-01,3,"[0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, -1..."
3,2013-01-01,4,"[0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
4,2013-01-01,5,"[0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."


In [31]:
# Pivot the data to be in the format (store number, date, features)
sales_data_pivoted = sales_features_store_df.pivot(index="store_nbr", columns="date", values="features")
sales_data_pivoted.head()

date,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05,2013-01-06,2013-01-07,2013-01-08,2013-01-09,2013-01-10,...,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15
store_nbr,,,,,,,,,,,,,,,,,,,,,
1,"[0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 1.0, 0.0, 0....","[7417.15, 93.14, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...","[5873.24, 92.97, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...","[5919.88, 93.12, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...","[6318.79, 93.12, 0, 1, 0.0, 0.0, 0.0, 1.0, 0.0...","[2199.09, 93.12, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...","[6150.92, 93.2, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[5597.18, 93.21, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...","[6808.58, 93.08, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...","[4757.71, 93.81, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...",...,"[4409.25, 49.57, 29, 0, 0.0, 0.0, 0.0, 1.0, 0....","[10538.14, 49.37, 44, 0, 0.0, 0.0, 0.0, 1.0, 0...","[11070.76, 49.07, 35, 0, 0.0, 0.0, 0.0, 1.0, 0...","[12583.65, 49.59, 212, 0, 0.0, 0.0, 0.0, 1.0, ...","[10834.58, 48.54, 31, 1, 0.0, 0.0, 0.0, 1.0, 0...","[5403.74, 48.81, 38, 1, 0.0, 0.0, 0.0, 1.0, 0....","[7543.62, 48.81, 28, 0, 0.0, 0.0, 0.0, 1.0, 0....","[3902.78, 48.81, 19, 0, 0.0, 0.0, 0.0, 1.0, 0....","[10433.31, 47.59, 30, 0, 0.0, 0.0, 0.0, 1.0, 0...","[9823.55, 47.57, 29, 0, 0.0, 0.0, 0.0, 1.0, 0...."
2,"[0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 1.0, 0.0, 0....","[10266.72, 93.14, 0, 0, 0.0, 0.0, 0.0, 1.0, 0....","[7612.54, 92.97, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...","[7526.38, 93.12, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...","[9539.57, 93.12, 0, 1, 0.0, 0.0, 0.0, 1.0, 0.0...","[10322.39, 93.12, 0, 0, 0.0, 0.0, 0.0, 1.0, 0....","[6675.97, 93.2, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[6109.57, 93.21, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...","[6740.52, 93.08, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...","[5565.04, 93.81, 0, 0, 0.0, 0.0, 0.0, 1.0, 0.0...",...,"[16987.57, 49.57, 43, 0, 0.0, 0.0, 0.0, 1.0, 0...","[12382.45, 49.37, 39, 0, 0.0, 0.0, 0.0, 1.0, 0...","[11304.55, 49.07, 38, 0, 0.0, 0.0, 0.0, 1.0, 0...","[12615.75, 49.59, 213, 0, 0.0, 0.0, 0.0, 1.0, ...","[11317.83, 48.54, 26, 1, 0.0, 0.0, 0.0, 1.0, 0...","[13081.77, 48.81, 57, 1, 0.0, 0.0, 0.0, 1.0, 0...","[12382.42, 48.81, 33, 0, 0.0, 0.0, 0.0, 1.0, 0...","[13766.82, 48.81, 32, 0, 0.0, 0.0, 0.0, 1.0, 0...","[12211.46, 47.59, 36, 0, 0.0, 0.0, 0.0, 1.0, 0...","[11196.26, 47.57, 33, 0, 0.0, 0.0, 0.0, 1.0, 0..."
3,"[0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, -1...","[24060.35, 93.14, 0, 0, 0.0, 0.0, 0.0, 0.0, 0....","[18570.75, 92.97, 0, 0, 0.0, 0.0, 0.0, 0.0, 0....","[17392.1, 93.12, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0...","[22700.87, 93.12, 0, 1, 0.0, 0.0, 0.0, 0.0, 0....","[25343.62, 93.12, 0, 0, 0.0, 0.0, 0.0, 0.0, 0....","[17036.25, 93.2, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0...","[14220.42, 93.21, 0, 0, 0.0, 0.0, 0.0, 0.0, 0....","[14735.15, 93.08, 0, 0, 0.0, 0.0, 0.0, 0.0, 0....","[12222.91, 93.81, 0, 0, 0.0, 0.0, 0.0, 0.0, 0....",...,"[42005.37, 49.57, 43, 0, 0.0, 0.0, 0.0, 0.0, 0...","[32074.15, 49.37, 41, 0, 0.0, 0.0, 0.0, 0.0, 0...","[28450.84, 49.07, 40, 0, 0.0, 0.0, 0.0, 0.0, 0...","[32962.39, 49.59, 222, 0, 0.0, 0.0, 0.0, 0.0, ...","[26035.93, 48.54, 30, 1, 0.0, 0.0, 0.0, 0.0, 0...","[32885.76, 48.81, 67, 1, 0.0, 0.0, 0.0, 0.0, 0...","[31903.48, 48.81, 34, 0, 0.0, 0.0, 0.0, 0.0, 0...","[34845.12, 48.81, 36, 0, 0.0, 0.0, 0.0, 0.0, 0...","[32843.7, 47.59, 34, 0, 0.0, 0.0, 0.0, 0.0, 0....","[30185.6, 47.57, 33, 0, 0.0, 0.0, 0.0, 0.0, 0...."
4,"[0.0, 93.14, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[10200.08, 93.14, 0, 0, 0.0, 0.0, 0.0, 0.0, 0....","[7400.25, 92.97, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0...","[6902.05, 93.12, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0...","[9948.38, 93.12, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0...","[11679.69, 93.12, 0, 0, 0.0, 0.0, 0.0, 0.0, 0....","[6748.2, 93.2, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[6017.61, 93.21, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0...","[5867.88, 93.08, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0...","[4674.25, 93.81, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0...",...,"[16801.05, 49.57, 32, 0, 0.0, 0.0, 0.0, 0.0, 0...","[10214.14, 49.37, 29, 0

In [32]:
# Convert the data to an array
stacked_sales_data = np.array(sales_data_pivoted.values.tolist())
stacked_sales_data.shape

(54, 1684, 18)

In [33]:
# Split the data into test/train/val sets with a 80/10/10 split
n = stacked_sales_data.shape[1]
train_data = stacked_sales_data[:, :int(n*0.8), :]
test_data = stacked_sales_data[:, int(n*0.8):int(n*0.9), :]
val_data = stacked_sales_data[:, int(n*0.9):, :]
print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print(f"Validation data shape: {val_data.shape}")

Train data shape: (54, 1347, 18)
Test data shape: (54, 168, 18)
Validation data shape: (54, 169, 18)


In [34]:
# Get the mean and standard deviation for normalization
scaler = preprocessing.StandardScaler()

# Flatten the first 2 dimensions into (stores*instances, features)
train_data_2d = train_data.reshape(-1, train_data.shape[2])
test_data_2d = test_data.reshape(-1, test_data.shape[2])
val_data_2d = val_data.reshape(-1, val_data.shape[2])

# Scale just the continuous features
train_data_2d[:, :num_continuous_features] = scaler.fit_transform(train_data_2d[:, :num_continuous_features])
test_data_2d[:, :num_continuous_features] = scaler.transform(test_data_2d[:, :num_continuous_features])
val_data_2d[:, :num_continuous_features] = scaler.transform(val_data_2d[:, :num_continuous_features])

print(f"Train data shape (2D): {train_data_2d.shape}")
print(f"Test data shape (2D): {test_data_2d.shape}")
print(f"Validation data shape (2D): {val_data_2d.shape}")

# Reshape the data back to its original dimensions
train_data = train_data_2d.reshape(train_data.shape)
test_data = test_data_2d.reshape(test_data.shape)
val_data = val_data_2d.reshape(val_data.shape)

print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print(f"Validation data shape: {val_data.shape}")

global_mean = scaler.mean_
global_stddev = scaler.scale_

print(global_mean, global_stddev)

# Save mean and standard deviation arrays to S3
np.save(f"{local_path}/global_mean.npy", global_mean)
np.save(f"{local_path}/global_stddev.npy", global_stddev)

sagemaker_session.upload_data(f"{local_path}/global_mean.npy", bucket=bucket, key_prefix=train_prefix)
sagemaker_session.upload_data(f"{local_path}/global_stddev.npy", bucket=bucket, key_prefix=train_prefix)

Train data shape (2D): (72738, 18)
Test data shape (2D): (9072, 18)
Validation data shape (2D): (9126, 18)
Train data shape: (54, 1347, 18)
Test data shape: (54, 168, 18)
Validation data shape: (54, 169, 18)
[11066.08424029    72.62890566    23.37328494] [9928.63142484   26.6484683    43.2551771 ]


's3://sagemaker-us-east-1-342408968837/store-sales-forecasting/train/global_stddev.npy'

In [35]:
# Define a function that breaks the data into time series windows of a specified size
# input_seq_length: The length of the input window
# target_seq_length: The length of the target (prediction) window
# stride: How much the windows overlap. A stride of 1 means each set of input/targets is shifted 1 day forward from the last input/targets
def generate_windows(data, input_seq_length, target_seq_length, stride):
    windows = []
    targets = []
    num_days = data.shape[1]
    
    for i in range(0, num_days, stride):
        if (i+input_seq_length+target_seq_length) <= num_days:
            input_window_end = i + input_seq_length
            target_window_end = input_window_end + target_seq_length
            
            input_window = data[:, i:input_window_end, :]
            target_window = data[:, input_window_end:target_window_end, 0]
            
            windows.append(input_window)
            targets.append(target_window)
            
    return np.array(windows), np.array(targets)

In [36]:
# Generate windows for train/test/val sets
input_seq_length = 7
target_seq_length = 1
stride = 1

train_inputs, train_targets = generate_windows(train_data, input_seq_length, target_seq_length, stride)
print(f"Train inputs shape: {train_inputs.shape}")
print(f"Train targets shape: {train_targets.shape}")

continuous_train_inputs = train_inputs[:, :, :, :num_continuous_features]
categorical_train_inputs = train_inputs[:, :, :, num_continuous_features:]
print(f"Continuous train inputs shape: {continuous_train_inputs.shape}")
print(f"Categorical train inputs shape: {categorical_train_inputs.shape}\n")

test_inputs, test_targets = generate_windows(test_data, input_seq_length, target_seq_length, stride)
print(f"Test inputs shape: {test_inputs.shape}")
print(f"Test targets shape: {test_targets.shape}")

continuous_test_inputs = test_inputs[:, :, :, :num_continuous_features]
categorical_test_inputs = test_inputs[:, :, :, num_continuous_features:]
print(f"Continuous test inputs shape: {continuous_test_inputs.shape}")
print(f"Categorical test inputs shape: {categorical_test_inputs.shape}\n")

val_inputs, val_targets = generate_windows(val_data, input_seq_length, target_seq_length, stride)
print(f"Validation inputs shape: {val_inputs.shape}")
print(f"Validation inputs shape: {val_targets.shape}")

continuous_val_inputs = val_inputs[:, :, :, :num_continuous_features]
categorical_val_inputs = val_inputs[:, :, :, num_continuous_features:]
print(f"Continuous validation inputs shape: {continuous_val_inputs.shape}")
print(f"Categorical validation inputs shape: {categorical_val_inputs.shape}")

Train inputs shape: (1340, 54, 7, 18)
Train targets shape: (1340, 54, 1)
Continuous train inputs shape: (1340, 54, 7, 3)
Categorical train inputs shape: (1340, 54, 7, 15)

Test inputs shape: (161, 54, 7, 18)
Test targets shape: (161, 54, 1)
Continuous test inputs shape: (161, 54, 7, 3)
Categorical test inputs shape: (161, 54, 7, 15)

Validation inputs shape: (162, 54, 7, 18)
Validation inputs shape: (162, 54, 1)
Continuous validation inputs shape: (162, 54, 7, 3)
Categorical validation inputs shape: (162, 54, 7, 15)


In [37]:
# Save data splits locally
np.save(f"{local_path}/continuous_train_inputs.npy", continuous_train_inputs)
np.save(f"{local_path}/categorical_train_inputs.npy", categorical_train_inputs)
np.save(f"{local_path}/train_targets.npy", train_targets)

np.save(f"{local_path}/continuous_test_inputs.npy", continuous_test_inputs)
np.save(f"{local_path}/categorical_test_inputs.npy", categorical_test_inputs)
np.save(f"{local_path}/test_targets.npy", test_targets)

np.save(f"{local_path}/continuous_val_inputs.npy", continuous_val_inputs)
np.save(f"{local_path}/categorical_val_inputs.npy", categorical_val_inputs)
np.save(f"{local_path}/val_targets.npy", val_targets)

In [38]:
# Save data splits to S3
sagemaker_session.upload_data(f"{local_path}/continuous_train_inputs.npy", bucket=bucket, key_prefix=train_prefix)
sagemaker_session.upload_data(f"{local_path}/categorical_train_inputs.npy", bucket=bucket, key_prefix=train_prefix)
sagemaker_session.upload_data(f"{local_path}/train_targets.npy", bucket=bucket, key_prefix=train_prefix)

sagemaker_session.upload_data(f"{local_path}/continuous_test_inputs.npy", bucket=bucket, key_prefix=test_prefix)
sagemaker_session.upload_data(f"{local_path}/categorical_test_inputs.npy", bucket=bucket, key_prefix=test_prefix)
sagemaker_session.upload_data(f"{local_path}/test_targets.npy", bucket=bucket, key_prefix=test_prefix)

sagemaker_session.upload_data(f"{local_path}/continuous_val_inputs.npy", bucket=bucket, key_prefix=val_prefix)
sagemaker_session.upload_data(f"{local_path}/categorical_val_inputs.npy", bucket=bucket, key_prefix=val_prefix)
sagemaker_session.upload_data(f"{local_path}/val_targets.npy", bucket=bucket, key_prefix=val_prefix)

's3://sagemaker-us-east-1-342408968837/store-sales-forecasting/val/val_targets.npy'